In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
sales = pd.read_csv('../data/raw/sales_train.csv')
shops = pd.read_csv('../data/raw/shops.csv')
items = pd.read_csv('../data/raw/items.csv')
item_cat = pd.read_csv('../data/raw/item_categories.csv')

### Deduplicating

In [3]:
duplicates_samples = sales[sales.duplicated(subset=['date','shop_id','item_id'])].shape[0]
old_shape = sales.shape

We can regroup our samples and handle duplicate rows, replacing `item_count` with a sum and `item_price` with a median.

In [4]:
sales = sales.groupby(['date', 'date_block_num', 'shop_id', 'item_id']).agg({'item_price':'median', 'item_cnt_day':'sum'}).reset_index()

In [5]:
assert (old_shape[0] - sales.shape[0]) == duplicates_samples

### Dropping negative prices

In [6]:
sales = sales[sales['item_price'] > 0]

### Dealing with outliars

I decided to drop only the most obvious outliars, 1-2 samples having enormous prices and counts.

In [7]:
sales = sales[sales['item_price'] < sales['item_price'].max()]
sales = sales[sales['item_cnt_day'] < sales['item_cnt_day'].max()]

### Possible repetitions in shops:

Жуковский ул. Чкалова 39м? (*id 10*) - Жуковский ул. Чкалова 39м² (*id 11*)

РостовНаДону ТРК "Мегацентр Горизонт" (*id 39*) - РостовНаДону ТРК "Мегацентр Горизонт" Островной (*id 40*)

!Якутск Орджоникидзе, 56 фран (*id 0*) - Якутск Орджоникидзе, 56 (*id 57*)

!Якутск ТЦ "Центральный" фран (*id 1*) - Якутск ТЦ "Центральный" (*id 58*)

I suggest remapping sales' attribute `shop_id` for samples to reference only one shop of the pair above.

In [8]:
shops_mapping = {10: 11, 40: 39, 0: 57, 1: 58}
sales['shop_id'].replace(shops_mapping, inplace=True)

### Special characters in item names

In [9]:
items.head(5)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [10]:
items['item_name'] = items['item_name'].apply(lambda name: re.sub('^[\\\/^.*\[\]~!@#$%^&()_+={}|\:;“’<,>?฿]+', '', name))

In [11]:
items.head(5)

,item_name,item_id,item_category_id
0,ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,ABBYY FineReader 12 Professional Edition Full ...,1,76
2,В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,КОРОБКА (СТЕКЛО) D,4,40


In [12]:
items[items.duplicated(subset='item_name')]

,item_name,item_id,item_category_id
14690,МИХЕЙ И ДЖУМАНДЖИ Сука любовь,14690,55


In [13]:
items[items['item_name'] == 'МИХЕЙ И ДЖУМАНДЖИ  Сука любовь']

,item_name,item_id,item_category_id
12,МИХЕЙ И ДЖУМАНДЖИ Сука любовь,12,55
14690,МИХЕЙ И ДЖУМАНДЖИ Сука любовь,14690,55


In [14]:
items_mapping = {12: 14690}
sales['item_id'].replace(items_mapping, inplace=True)

`item_categories` is clean.

In [15]:
test = pd.read_csv('../data/raw/test.csv')
test['item_id'].replace(items_mapping, inplace=True)
test['shop_id'].replace(shops_mapping, inplace=True)

### Saving data after ETL to intermediate storage

In [16]:
sales.to_csv('../data/interim/sales_train_etl.csv', index=False)
shops.to_csv('../data/interim/shops_etl.csv', index=False)
items.to_csv('../data/interim/items_etl.csv', index=False)
item_cat.to_csv('../data/interim/item_categories_etl.csv', index=False)
test.to_csv('../data/interim/test_etl.csv', index=False)